In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [31]:
from typing import TypedDict

class State(TypedDict):
    task: str
    draft: str
    feedback: str
    final: str

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a skilled general purpose assistant that follows given tasks exactly and responds with brief responses."),
    ("user", "{user_input}")
])

chain = prompt | llm

In [48]:
# Nodes
def generator(state: State) -> State:
    prompt = f"""
    Help in completing the following task:

    Task:
    {state['task']}

    Current Draft:
    {state['draft']}

    Feedback:
    {state['feedback']}

    Instructions:
    - If there is no draft and no feedback, generate a clear and complete reponse to the task.
    - If there is a draft but no feedback, improve the draft as needed for clarity and quality.
    - If there is both a draft and a feedback, revise the draft by incorporating the feedback directly.
    - Always produce a single, improved draft as your output.
    """
    response = chain.invoke({"user_input": prompt})
    return {"draft": response.content.strip()}

def evaluator(state: State) -> State:
    prompt = f"""Evaluate the following draft based on the given task.
    If it is a writing task also check if the content is arranged by bullet points. Enforce this through feedback.
    If it meets the requirements, reply exactly 'APPROVED'.
    Otherise, provide constructive feedback for improvement.

    Task:
    {state['task']}

    Current Draft:
    {state['draft']}
    """

    response = chain.invoke({"user_input": prompt})
    print(f"""
===========================================================
----------------------[ DRAFT ]----------------------------
{state['draft']}

---------------------[ FEEDBACK ]--------------------------
{state['feedback']}
-----------------------------------------------------------
===========================================================
""")
    return {"feedback": response.content.strip()}

def decide(state: State) -> str:
    if "APPROVED" == state["feedback"].upper():
        print("\n", ">" * 20, state["feedback"], "<" * 20)
        return "approved"
    return "refine"

def finalize(state: State) -> State:
    return {"final": state["draft"]}

In [49]:
from langgraph.graph import StateGraph, START, END

# Build the graph
workflow = StateGraph(State)

workflow.add_node("generator", generator)
workflow.add_node("evaluator", evaluator)
workflow.add_node("finalize", finalize)

workflow.add_edge(START, "generator")
workflow.add_edge("generator", "evaluator")

workflow.add_conditional_edges(
    "evaluator",
    decide,
    {
        "approved": "finalize",
        "refine": "generator"
    }
)

workflow.add_edge("finalize", END)

In [50]:
# Compile graph
graph = workflow.compile()

In [51]:
# TEST
input_task = "Give an essay on how to manufacture a bicycle written by a 5 year old."
result = graph.invoke({"task": input_task, "draft": "", "feedback":""})

print("*" * 80)
print("\nFinal Answer:")
print(result["final"])


----------------------[ DRAFT ]----------------------------
Making a bike is super fun! First, you need a big, strong frame. It's like the body of the bike, where everything else sticks. My daddy says it's made of metal, but I think it looks like a big triangle.

Then, you need two round wheels! One for the front and one for the back. You put them on the frame so they can spin, spin, spin! And don't forget the tires, they are like big black doughnuts that go on the wheels so the bike can roll bumpy or smooth.

Next, you need a comfy seat for your bottom. You stick it on a pole and put it in the frame. And two handle-bars so you can steer, steer, steer! They go on the front, right above the front wheel. They are good for holding on tight!

Then, the pedals! They are special because your feet push them around and around, and that makes the bike go zoom! You put them near the bottom, sticking out from the middle.

Sometimes, bikes need little chains to help the pedals talk to the wheels.